In [ ]:
!pip install transformers

In [ ]:
from PIL import Image
from transformers import pipeline
import torch
import os
import cv2
from tqdm import tqdm
from google.colab.patches import cv2_imshow

vqa_pipeline = pipeline("visual-question-answering")

No model was supplied, defaulted to dandelin/vilt-b32-finetuned-vqa and revision 4355f59 (https://huggingface.co/dandelin/vilt-b32-finetuned-vqa).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
!mkdir visualqa_output

In [ ]:
directory_path = '/content/videos'
directory_files = os.listdir(directory_path)
#video_list = ['1.mp4','2.mp4','3.mp4','4.mp4']
#The input and outputAppend sentences must match.
inputSentences = ["In which area did this accident happen?", "What is the time of the day when this accident happened?", "What is the weather condition in this accident?",
                  "What is the season of",
                  "How many vehicles are in involved in this accident?", "What type of vehicles are in the provided image?",
                  "What is the damaged vehicle in the provided image?",
                  "Which side of vehicle is damaged in the provided image? Front or Back?"]
outputAppendText = ["The accident occurred on ", "The accident took place at ",
                    "The accident happened under weather condition, ", "The incident took place during ",
                    "The number of vehicles are ",
                    "The road had ", "The vehicle involved in the accident was ",
                    "The vehicle sustained damage on its "]
#outputSentence = []
for video in directory_files:
  #outputText = []
  outputText = set()
  array_of_dict = [{},{},{},{}]
  print(video)
  # Create a VideoCapture object and read from input file
  cap = cv2.VideoCapture("/content/videos/" + video)

  # Get the frame rate of the video
  fps = cap.get(cv2.CAP_PROP_FPS)

  # Define the desired frame interval (e.g., every second)
  frame_interval = int(fps)  # Extract one frame per second

  # Get the total number of frames in the video
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  # Create a tqdm progress bar
  progress_bar = tqdm(total=total_frames, desc='Processing Frames', unit='frame')

  if (cap.isOpened()== False):
  # Give a error message
    print("Error opening video file")

  progress = 0
  frame_count = 0
  final_caption = ""
  final_frames_output = ""
  # Read until video is completed
  while(cap.isOpened()):

  # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        progress_bar.update(1)
        progress = progress + 1
        if(progress > total_frames):
          break
        frame_count = frame_count + 1
        if frame_count%(frame_interval) == 0:
          # process image to text
          device = "cuda" if torch.cuda.is_available() else "cpu"
          #cv2_imshow(frame)
          #outputFrameCount = 0
          frame_output = ""
          for i in range(0, len(inputSentences)):
            # Convert from OpenCV image (BGR) to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Convert the frame to a PIL Image
            pil_image = Image.fromarray(frame_rgb)
            outputs = vqa_pipeline(pil_image, inputSentences[i], top_k=1)
            #print(processor.decode(outputs[0], skip_special_tokens=True))
            outputLabel = outputs[0].get("answer")
            if i <= 3:
              if outputLabel not in array_of_dict[i]:
                array_of_dict[i][outputLabel] = 0
              array_of_dict[i][outputLabel] = array_of_dict[i][outputLabel] + 1
            else:
              if outputLabel == 'none' or outputLabel == 'no' or outputLabel == 'no vehicle' or outputLabel == 0 or outputLabel == '0' or outputLabel == 'no car' or outputLabel == 1 or outputLabel == '1' or outputLabel == 'unknown':
                print("")
              else:
                outputText.add(outputAppendText[i] + outputLabel + ".")
                frame_output = frame_output + outputAppendText[i] + outputLabel + "."
            #if(outputFrameCount <= 1):
              #if not outputText:
                #new_dict = {outputLabel: 1}
              #else:
                #new_dict = {outputLabel: 1}
              #outputText.append(new_dict)
            #else:
              #for dictionary in outputText:
                #if outputLabel not in dictionary:
                  #dictionary[outputLabel] = 0
                #dictionary[outputLabel] = dictionary[outputLabel] + 1
            #final_caption = final_caption + generated_caption
            cv2.waitKey(0)
          final_frames_output = final_frames_output + "frame " + str(frame_count) + ", " + frame_output
      # Press Q on keyboard to exit
          if cv2.waitKey(25) & 0xFF == ord('q'):
              break

  # Break the loop
      else:
          break
  #print(final_caption)
  #Iterate through each dictionary and find the key with the highest value
  if array_of_dict:
    outputLabelText = [max(d, key=d.get) for d in array_of_dict]
    for text, appendText in zip(outputLabelText, outputAppendText):
      outputText.add(appendText + text + ".")
      final_frames_output = final_frames_output + "common for all frames, " + appendText + text + "."
  # When everything done, release
  # the video capture object
  cap.release()

  # Closes all the frames
  cv2.destroyAllWindows()
  # Iterate through the dictionaries in the input list
  #for dictionary in outputText:
      # Remove the "none" key if it exists
  #    dictionary.pop("none", None)

  # Iterate through each dictionary and find the key with the highest value
  #outputLabelText = [max(d, key=d.get) if d else None for d in outputText]
  #for text, appendText in zip(outputLabelText, outputAppendText):
  #  outputSentence.append(appendText + text)
  #print('\n')
  #print(''.join(outputSentence))
  print(outputText)
  f = open("/content/visualqa_output/" + video.split(".")[0] + ".txt", "w")
  f.write(final_frames_output)
  f.close()
  print(final_frames_output)

CLIP_322.mp4


Processing Frames:   2%|▏         | 28/1348 [00:00<00:27, 47.72frame/s]

Processing Frames:   4%|▍         | 57/1348 [00:16<06:40,  3.23frame/s]

Processing Frames:   5%|▌         | 69/1348 [00:23<08:27,  2.52frame/s]

Processing Frames:   8%|▊         | 114/1348 [00:32<04:52,  4.21frame/s]

...

Processing Frames:  86%|████████▋ | 769/890 [04:12<00:47,  2.53frame/s]

Processing Frames:  89%|████████▉ | 793/890 [04:21<00:37,  2.61frame/s]

Processing Frames:  92%|█████████▏| 817/890 [04:28<00:25,  2.87frame/s]

Processing Frames:  94%|█████████▍| 841/890 [04:37<00:17,  2.82frame/s]

{'The vehicle sustained damage on its back.', 'The number of vehicles are 3.', 'The vehicle sustained damage on its front.', 'The number of vehicles are 2.', 'The vehicle involved in the accident was sign.', 'The road had trucks.', 'The accident happened under weather condition, cloudy.', 'The accident took place at evening.', 'The accident occurred on highway.', 'The vehicle involved in the accident was motorcycle.', 'The road had motorcycles.', 'The vehicle sustained damage on its left.', 'The vehicle involved in the accident was car.', 'The number of vehicles are 4.', 'The road had cars.', 'The incident took place during summer.'}
frame 24, The road had cars.The vehicle involved in the accident was car.The vehicle sustained damage on its front.frame 48, The road had cars.The vehicle involved in the accident was car.The vehicle sustained damage on its back.frame 72, The number of vehicles are 3.The road had cars.The vehicle involved in the accident was car.The vehicle sustained damag

In [ ]:
!zip -r /content/visualqa_output.zip /content/visualqa_output/

  adding: content/visualqa_output/ (stored 0%)
  adding: content/visualqa_output/CLIP_298.txt (deflated 87%)
  adding: content/visualqa_output/CLIP_322.txt (deflated 92%)
  adding: content/visualqa_output/CLIP_315.txt (deflated 91%)
  adding: content/visualqa_output/CLIP_286.txt (deflated 91%)
  adding: content/visualqa_output/CLIP_341.txt (deflated 92%)
  adding: content/visualqa_output/CLIP_261.txt (deflated 92%)
  adding: content/visualqa_output/CLIP_289.txt (deflated 92%)
  adding: content/visualqa_output/CLIP_283.txt (deflated 90%)
  adding: content/visualqa_output/CLIP_324.txt (deflated 91%)
  adding: content/visualqa_output/CLIP_304.txt (deflated 87%)
  adding: content/visualqa_output/CLIP_264.txt (deflated 90%)
  adding: content/visualqa_output/CLIP_301.txt (deflated 91%)
  adding: content/visualqa_output/CLIP_308.txt (deflated 92%)
  adding: content/visualqa_output/CLIP_312.txt (deflated 89%)
  adding: content/visualqa_output/CLIP_336.txt (deflated 91%)
  adding: content/visua